# Food Reviews Exploration

### Description

Personal project to explore natural language and user behavior analysis methods using Python, Power BI and LLM aided analysis of clusters. The dataset used consists of 568,454 reviews on Amazon fine foods for 74,258 different products from Oct 1999 to Oct 2012. Reviews include product and user information, ratings, and a plain text review. 

| Variable | Description |
| :- | :- |
| Id | Assigned number for product. |
| ProductId | Unique identifier for the product |
| UserId | Unique identifier for the user |
| ProfileName | Profile name of the user |
| HelpfulnessNumerator | Number of users who found the review helpful |
| HelpfulnessDenominator | Number of users who indicated review helpfulness |
| Score | Rating between 1 and 5  |
| Time | Timestamp for the review  |
| Summary | Brief summary of the review |
| Text | Text of the review  |

### Data cleaning

- 2 rows with invalid numerators were deleted.
- The lack of product or food categories limited the analysis; Product IDs no longer exist in the Amazon catalogue so they could not be sourced externally. Analysis focused on themes and sentiment.
- 30,7% of texts are duplicates. A manual sample reviews showed probable relation to the scraping process or spam posting. These rows were dropped to finetune the text analysis.
- Two reviews with over 15,000 characters were discarded as outliers.

### Analysis objectives: 

- Review engagement motivation.
- Negative vs positive reviews main motifs.
- Helpfulness trends (What makes a review helpful?).
- Analize yearly and monthly patterns.
- Recurrent vs one-time user trends.

### EDA

- Reviews increased exponentially up to 2012, peaking by 2021, which correlates to product variety increase (Unique Product IDs). October sees the highest average review count, while November sees the lowest.
- Score: 5 star reviews dominate the dataset consistently, followed by 1 start reviews.
- Helpfulness: Helpful votes grew 5 times faster and reach 3 times more counts than Non Helpful votes. 
- Users: 80% of users reviewed once; 0,05%% posted >50 reviews. Excessive reviewers gathered more positive scores; based on reading reviewed, many are potential bots or spam users.
- Engagement and themes: Reviews cluster around experience, emotion, health, practical value. Most helpful reviews further highlight warnings, ingredients and quality.
- Longer texts tend to have 5 starts, and 500-1,000 character lengths clutter most helpful reviews, suggesting 20–60 seconds reading time is optimal. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

In [41]:
df=pd.read_csv("foodreviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df[['ProductId', 'Id']].nunique()

In [ ]:
df = df.drop_duplicates(subset='Text')
df[['ProductId', 'Id']].nunique()

In [ ]:
invalid_rows = df[df['HelpfulnessNumerator'] > df['HelpfulnessDenominator']]
print(f"Invalid rows found: {len(invalid_rows)}")

In [ ]:
df = df[df['HelpfulnessNumerator'] <= df['HelpfulnessDenominator']]

In [ ]:
df['Date'] = pd.to_datetime(df['Time'], unit='s')

df['Year'] = pd.to_datetime(df['Date']).dt.year

df['Month'] = pd.to_datetime(df['Date']).dt.month

In [ ]:
user_counts = df['UserId'].value_counts()
total_users = user_counts.shape[0]

unique_users = (user_counts == 1).sum() / total_users * 100
recurrent_users = ((user_counts > 1) & (user_counts <= 50)).sum()  / total_users * 100
excessive_users = (user_counts > 50).sum()  / total_users * 100

print(f"Unique users (reviewed once): {unique_users}")
print(f"Recurrent users (multiple reviews): {recurrent_users}")
print(f"Excessive users (>30 reviews): {excessive_users}")

In [ ]:
unique_prod_per_year = df.groupby('Year')['ProductId'].nunique()

plt.figure(figsize=(8,5))
unique_prod_per_year.plot(kind='bar')
plt.title('Unique ProductId Count by Year')
plt.xlabel('Year')
plt.ylabel('Unique ProductId Count')
plt.show()

In [ ]:
df['UserReviewCount'] = df.groupby('UserId')['UserId'].transform('count')
df['UserFrequency'] = np.where(df['UserReviewCount'] > 50, 'Excessive', 'Expected')

df_filtered_excessive = df[df['UserFrequency'] == 'Excessive']

df_filtered_excessive['Score'].value_counts()

In [ ]:
df['HelpfulnessRate'] = df['HelpfulnessNumerator'] / df['HelpfulnessDenominator']
df['HelpfulnessRate'].value_counts()


In [ ]:
df['NonHelpfulnessNumerator'] = df['HelpfulnessDenominator'] - df['HelpfulnessNumerator']

In [ ]:
df['Text'].duplicated().sum()

In [ ]:
df['UserId'].nunique()

In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(data=df, x='HelpfulnessDenominator', bins=np.arange(0, 200, 5), multiple='stack')
plt.grid(True)
plt.title('Helpfulness Participation Histogram');

In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(data=df[df['HelpfulnessDenominator'] != 0], x='HelpfulnessRate', bins=np.arange(0, 1, 0.05), multiple='stack')
plt.title('Helpfulness Rate Histogram');

In [ ]:
df[['HelpfulnessNumerator', 'NonHelpfulnessNumerator']].describe()

In [ ]:
plt.figure(figsize=(20, 6))
sns.boxplot(data=df[['HelpfulnessNumerator', 'NonHelpfulnessNumerator']], orient='h')
plt.title('Boxplots of Helpfulness Reactions')
plt.show()

In [ ]:
filtered_help = df[df['HelpfulnessNumerator']<150]

plt.figure(figsize=(20, 6))
sns.boxplot(data=filtered_help[['HelpfulnessNumerator', 'NonHelpfulnessNumerator']], orient='h')
plt.title('Boxplots of Helpfulness Reactions')
plt.show()

In [ ]:
plt.figure(figsize=(4, 3))
sns.countplot(data=df, x='Score')
plt.title('Count of Scores')
plt.show()

In [ ]:
grouped_score = df.groupby('Score')[['HelpfulnessNumerator', 'NonHelpfulnessNumerator']].mean().reset_index()
print(grouped_score)

In [ ]:
yearly_sum = df[['Year', 'HelpfulnessNumerator', 'NonHelpfulnessNumerator']].groupby('Year').sum()

plt.figure(figsize=(10, 6))
plt.plot(yearly_sum.index, yearly_sum['HelpfulnessNumerator'], marker='o', label='HelpfulnessNumerator')
plt.plot(yearly_sum.index, yearly_sum['NonHelpfulnessNumerator'], marker='o', label='NonHelpfulnessNumerator')

plt.xlabel('Year')
plt.ylabel('Sum')
plt.title('Yearly Sum of Helpfulness Metrics')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='Year', hue='Score')
plt.title('Count of Reviews by Year')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='Month', hue='Score')
plt.title('Count of Reviews by Month')
plt.xticks(rotation=45)
plt.show()

In [ ]:
stopwords = set(STOPWORDS)
custom_stopwords = set(STOPWORDS)
custom_stopwords.add('br')

pos_joined_text = ' '.join(df[df['Score'] == 5]['Text'].astype(str))

pos_wordcloud = WordCloud(width=1000, height=400, background_color='white', colormap='Greens', stopwords=custom_stopwords, collocations=False).generate(pos_joined_text)

plt.figure(figsize=(15, 5))
plt.imshow(pos_wordcloud, interpolation='bilinear')
plt.title('Wordcloud for reviews on positive scores (5)', fontsize=12)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
stopwords = set(STOPWORDS)
neg_joined_text = ' '.join(df[df['Score'] == 1]['Text'].astype(str))

neg_wordcloud = WordCloud(width=1000, height=400, background_color='white', colormap='Reds', stopwords=custom_stopwords, collocations=False).generate(neg_joined_text)

plt.figure(figsize=(15, 5))
plt.imshow(neg_wordcloud, interpolation='bilinear')
plt.title('Wordcloud for reviews on negative scores (1)', fontsize=12)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
tophelpful_joined_text = ' '.join(df[df['HelpfulnessNumerator'] > 200]['Text'].astype(str))

helpful_wordcloud = WordCloud(width=1000, height=400, background_color='white', colormap='Blues', stopwords=custom_stopwords, collocations=False).generate(tophelpful_joined_text)

plt.figure(figsize=(15, 5))
plt.imshow(helpful_wordcloud, interpolation='bilinear')
plt.title('Wordcloud for Reviews with > 200 Helpful Reactions', fontsize=12)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
df[['HelpfulnessNumerator', 'ProductId', 'Summary', 'Text']].sort_values(by='HelpfulnessNumerator', ascending=False).head()

In [ ]:
df['TextLen'] = df['Text'].str.len()

plt.figure(figsize=(15, 3))
sns.boxplot(x=df['TextLen'])
plt.xlabel('Text Length')
plt.title('Boxplot of Text Length')
plt.show()

In [ ]:
df_lenfiltered = df[df['Text'].str.len() <= 15000]
plt.figure(figsize=(15, 6))
plt.scatter(df_lenfiltered['TextLen'], df_lenfiltered['HelpfulnessNumerator'], alpha=0.3, s=10)
plt.title('Length vs. Helpfulness')
plt.xlabel('Length of Review Text (characters)')
plt.ylabel('Helpfulness Numerator')
plt.grid(True)

In [ ]:
plt.figure(figsize=(10, 3))
plt.scatter(df_lenfiltered['TextLen'], df_lenfiltered['Score'], alpha=0.3, s=10)
plt.title('Length vs. Score')
plt.xlabel('Length of Review Text (characters)')
plt.ylabel('Score')
plt.grid(True)

In [ ]:
def get_top_words(texts, top_n=20):
    vectorizer = CountVectorizer(stop_words='english', max_features=1000)
    X = vectorizer.fit_transform(texts)
    
    word_counts = X.toarray().sum(axis=0)
    vocab = vectorizer.get_feature_names_out()
    
    word_freq = pd.Series(word_counts, index=vocab)
    return word_freq.sort_values(ascending=False).head(top_n)

# Filter your datasets
texts_positive = df.loc[df['Score'] == 5, 'Summary'].dropna()
texts_negative = df.loc[df['Score'] == 1, 'Summary'].dropna()
texts_helpful = df.loc[df['HelpfulnessNumerator'] > 10, 'Summary'].dropna()

# Get top words
print("Top words in positive reviews:")
print(get_top_words(texts_positive))

print("\nTop words in negative reviews:")
print(get_top_words(texts_negative))

print("\nTop words in helpful reviews:")
print(get_top_words(texts_helpful))

print("\nTop words in most reviewd product:")
print(get_top_words(text_top_product))

In [ ]:
def cluster_and_show_top_words(texts, n_clusters=5, max_features=500, top_n=10):
    vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english')
    X = vectorizer.fit_transform(texts)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(X)
    
    terms = vectorizer.get_feature_names_out()
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    
    for i in range(n_clusters):
        top_words = [terms[ind] for ind in order_centroids[i, :top_n]]
        print(f"Cluster {i} top words: {', '.join(top_words)}")
    print('-' * 40)

# Filter your data
summary_positive = df.loc[df['Score'] == 5, 'Summary'].dropna()
summary_negative = df.loc[df['Score'] == 1, 'Summary'].dropna()
text_top_product = df.loc[df['ProductId'] == 'B007JFMH8M', 'Text'].dropna()

print("Positive Reviews Clusters:")
cluster_and_show_top_words(summary_positive)

print("Negative Reviews Clusters:")
cluster_and_show_top_words(summary_negative)

print("Negative Reviews Clusters:")
cluster_and_show_top_words(text_top_product)

In [ ]:
df[df['ProductId'] == df['ProductId'].value_counts().idxmax()][['ProductId', 'Summary', 'Date']].head(3)

In [ ]:
top_product = df[df['ProductId'] == 'B007JFMH8M']
top_product.shape

In [ ]:
top_product[['Text', 'Date']].head()

In [ ]:
def categorize_sentiment_detailed(text):
    polarity = TextBlob(text).sentiment.polarity
    
    if polarity >= 0.6:
        return "Strongly Positive"
    elif polarity > 0.1:
        return "Positive"
    elif polarity >= -0.1 and polarity <= 0.1:
        return "Neutral"
    elif polarity < -0.1 and polarity >= -0.6:
        return "Negative"
    else:
        return "Strongly Negative"
    
df['SentimentText'] = df['Text'].apply(categorize_sentiment_detailed)

In [ ]:
df.to_csv("FoodReviews_V2.csv", index=False)